In [4]:
from datetime import date, timedelta
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
import matplotlib as plp
import lightgbm as lgb

In [21]:
df_train = pd.read_csv(
    '../Data/train_set.csv', usecols=[1, 2, 3, 4, 5],
    converters={'unit_sales': lambda u: np.log1p(
        float(u)) if float(u) > 0 else 0},
    parse_dates=["date"],dtype={'onpromotion': bool}
)

In [9]:
df_test = pd.read_csv(
    "../Data/test.csv", usecols=[0, 1, 2, 3, 4],
    dtype={'onpromotion': bool},
    parse_dates=["date"]  # , date_parser=parser
).set_index(
    ['store_nbr', 'item_nbr', 'date']
)

In [11]:
items = pd.read_csv(
    "../Data/items.csv",
).set_index("item_nbr") #In order to give weight to item perishable

In [23]:
promo_train = df_train.set_index(
    ["store_nbr", "item_nbr", "date"])[["onpromotion"]].unstack(
        level=-1).fillna(False)

In [25]:
promo_train.columns = promo_train.columns.get_level_values(1)

In [28]:
promo_test = df_test[["onpromotion"]].unstack(level=-1).fillna(False)
promo_test.columns = promo_test.columns.get_level_values(1)

In [33]:
promo_test = promo_test.reindex(promo_2017_train.index).fillna(False)
promo_2017 = pd.concat([promo_train, promo_test], axis=1)
del promo_test, promo_train

In [38]:
df_train = df_train.set_index(
    ["store_nbr", "item_nbr", "date"])[["unit_sales"]].unstack(
        level=-1).fillna(0)
df_train.columns = df_train.columns.get_level_values(1)

KeyError: "['store_nbr', 'item_nbr', 'date']"

In [39]:
items = items.reindex(df_train.index.get_level_values(1))

In [40]:
def get_timespan(df, dt, minus, periods):
    return df[
        pd.date_range(dt - timedelta(days=minus), periods=periods)
    ]

In [46]:
def prepare_dataset(t2017, is_train=True):
    X = pd.DataFrame({
        "mean_3_2017": get_timespan(df_train, t2017, 3, 3).mean(axis=1).values,
        "mean_7_2017": get_timespan(df_train, t2017, 7, 7).mean(axis=1).values,
        "mean_14_2017": get_timespan(df_train, t2017, 14, 14).mean(axis=1).values,
        "mean_16_2017": get_timespan(df_train, t2017, 16, 16).mean(axis=1).values,
        "promo_14_2017": get_timespan(promo_2017, t2017, 14, 14).sum(axis=1).values  
    })
    for i in range(16):
        X["promo_{}".format(i)] = promo_2017[t2017 + timedelta(days=i)].values.astype(np.uint8)
    if is_train:
        y = df_train[
            pd.date_range(t2017, periods=16)
        ].values
        return X, y
    return X

In [53]:
print("Preparing dataset...")
t2017 = date(2017, 5, 16)
X_l, y_l = [], []
for i in range(9):
    delta = timedelta(days=7 * i)
    X_tmp, y_tmp = prepare_dataset(t2017 + delta)
    X_l.append(X_tmp)
    y_l.append(y_tmp)
X_train = pd.concat(X_l, axis=0)
y_train = np.concatenate(y_l, axis=0)
del X_l, y_l

Preparing dataset...


In [54]:
X_val, y_val = prepare_dataset(date(2017, 7, 23))

In [55]:
X_test = prepare_dataset(date(2017, 8, 16), is_train=False)

In [56]:
print("Training and predicting models...")
params = {
    'num_leaves': 80,
    'objective': 'regression',
    'min_data_in_leaf': 200,
    'learning_rate': 0.02,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.7,
    'bagging_freq': 1,
    'metric': 'l2',
    'num_threads': 16
}

MAX_ROUNDS = 5000
val_pred = []
test_pred = []
cate_vars = []
for i in range(16):
    print("=" * 50)
    print("Step %d" % (i+1))
    print("=" * 50)
    dtrain = lgb.Dataset(
        X_train, label=y_train[:, i],
        categorical_feature=cate_vars,
        weight=pd.concat([items["perishable"]] * 9) * 0.25 + 1
    )
    
    dval = lgb.Dataset(
        X_val, label=y_val[:, i], reference=dtrain,
        categorical_feature=cate_vars,
        weight=items["perishable"] * 0.25 + 1
    )
    
    bst = lgb.train(
        params, dtrain, num_boost_round=MAX_ROUNDS,
        valid_sets=[dtrain, dval], early_stopping_rounds=50, verbose_eval=50
    )
    
    print("\n".join(("%s: %.2f" % x) for x in sorted(
        zip(X_train.columns, bst.feature_importance("gain")),
        key=lambda x: x[1], reverse=True
    )))
    val_pred.append(bst.predict(
        X_val, num_iteration=bst.best_iteration or MAX_ROUNDS))
    test_pred.append(bst.predict(
        X_test, num_iteration=bst.best_iteration or MAX_ROUNDS))

print("Validation mse:", mean_squared_error(
    y_val, np.array(val_pred).transpose()))

Training and predicting models...
Step 1


/Users/arkarmin/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1186: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 50 rounds.
[50]	training's l2: 0.446218	valid_1's l2: 0.577263
[100]	training's l2: 0.355342	valid_1's l2: 0.457986
[150]	training's l2: 0.341092	valid_1's l2: 0.432492
[200]	training's l2: 0.337355	valid_1's l2: 0.424829
[250]	training's l2: 0.335835	valid_1's l2: 0.421992
[300]	training's l2: 0.334911	valid_1's l2: 0.420669
[350]	training's l2: 0.334195	valid_1's l2: 0.419768
[400]	training's l2: 0.333731	valid_1's l2: 0.419385
[450]	training's l2: 0.333336	valid_1's l2: 0.419186
[500]	training's l2: 0.333009	valid_1's l2: 0.419049
[550]	training's l2: 0.332717	valid_1's l2: 0.418956
[600]	training's l2: 0.332424	valid_1's l2: 0.418879
[650]	training's l2: 0.332168	valid_1's l2: 0.4188
[700]	training's l2: 0.331926	valid_1's l2: 0.418756
[750]	training's l2: 0.33169	valid_1's l2: 0.41873
Early stopping, best iteration is:
[748]	training's l2: 0.3317	valid_1's l2: 0.41873
mean_7_2017: 8995277.11
mean_14_2017: 6833645.58
mean_16_2017: 

/Users/arkarmin/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1186: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 50 rounds.
[50]	training's l2: 0.45031	valid_1's l2: 0.486588
[100]	training's l2: 0.355294	valid_1's l2: 0.393491
[150]	training's l2: 0.340195	valid_1's l2: 0.377785
[200]	training's l2: 0.336444	valid_1's l2: 0.374277
[250]	training's l2: 0.334934	valid_1's l2: 0.372954
[300]	training's l2: 0.333964	valid_1's l2: 0.372072
[350]	training's l2: 0.333227	valid_1's l2: 0.371589
[400]	training's l2: 0.332755	valid_1's l2: 0.371225
[450]	training's l2: 0.332322	valid_1's l2: 0.371011
[500]	training's l2: 0.331948	valid_1's l2: 0.370739
[550]	training's l2: 0.331629	valid_1's l2: 0.370647
[600]	training's l2: 0.33135	valid_1's l2: 0.370542
[650]	training's l2: 0.33108	valid_1's l2: 0.370459
[700]	training's l2: 0.330794	valid_1's l2: 0.370331
[750]	training's l2: 0.330539	valid_1's l2: 0.37029
[800]	training's l2: 0.330312	valid_1's l2: 0.370242
[850]	training's l2: 0.330083	valid_1's l2: 0.3702
[900]	training's l2: 0.329855	valid_1's l2: 

...

Training until validation scores don't improve for 50 rounds.
[50]	training's l2: 0.515899	valid_1's l2: 0.589267
[100]	training's l2: 0.428443	valid_1's l2: 0.487673
[150]	training's l2: 0.413365	valid_1's l2: 0.467036
[200]	training's l2: 0.409164	valid_1's l2: 0.46091
[250]	training's l2: 0.407045	valid_1's l2: 0.458561
[300]	training's l2: 0.405632	valid_1's l2: 0.457416
[350]	training's l2: 0.404583	valid_1's l2: 0.456875
[400]	training's l2: 0.403891	valid_1's l2: 0.456494
[450]	training's l2: 0.403305	valid_1's l2: 0.4562
[500]	training's l2: 0.402796	valid_1's l2: 0.456079
[550]	training's l2: 0.402357	valid_1's l2: 0.455926
[600]	training's l2: 0.401983	valid_1's l2: 0.455805
[650]	training's l2: 0.40163	valid_1's l2: 0.455692
[700]	training's l2: 0.401281	valid_1's l2: 0.455618
[750]	training's l2: 0.400979	valid_1's l2: 0.455524
[800]	training's l2: 0.400673	valid_1's l2: 0.455429
[850]	training's l2: 0.400371	valid_1's l2: 0.455366
[900]	training's l2: 0.400073	valid_1's l2

/Users/arkarmin/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1186: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 50 rounds.
[50]	training's l2: 0.498692	valid_1's l2: 0.681944
[100]	training's l2: 0.414732	valid_1's l2: 0.570097
[150]	training's l2: 0.399315	valid_1's l2: 0.543733
[200]	training's l2: 0.394863	valid_1's l2: 0.535061
[250]	training's l2: 0.392758	valid_1's l2: 0.53166
[300]	training's l2: 0.391352	valid_1's l2: 0.529883
[350]	training's l2: 0.39038	valid_1's l2: 0.528954
[400]	training's l2: 0.389683	valid_1's l2: 0.528529
[450]	training's l2: 0.389154	valid_1's l2: 0.528179
[500]	training's l2: 0.388703	valid_1's l2: 0.527866
[550]	training's l2: 0.38829	valid_1's l2: 0.52766
[600]	training's l2: 0.387928	valid_1's l2: 0.527515
[650]	training's l2: 0.387574	valid_1's l2: 0.527318
[700]	training's l2: 0.387249	valid_1's l2: 0.52711
[750]	training's l2: 0.386939	valid_1's l2: 0.527032
[800]	training's l2: 0.386653	valid_1's l2: 0.526912
[850]	training's l2: 0.386382	valid_1's l2: 0.526824
[900]	training's l2: 0.386119	valid_1's l2:

/Users/arkarmin/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1186: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 50 rounds.
[50]	training's l2: 0.496176	valid_1's l2: 0.53969
[100]	training's l2: 0.405356	valid_1's l2: 0.450953
[150]	training's l2: 0.389516	valid_1's l2: 0.434446
[200]	training's l2: 0.385472	valid_1's l2: 0.430244
[250]	training's l2: 0.383709	valid_1's l2: 0.428609
[300]	training's l2: 0.382519	valid_1's l2: 0.427593
[350]	training's l2: 0.381662	valid_1's l2: 0.427205
[400]	training's l2: 0.381024	valid_1's l2: 0.426957
[450]	training's l2: 0.380513	valid_1's l2: 0.426701
[500]	training's l2: 0.380078	valid_1's l2: 0.426505
[550]	training's l2: 0.379693	valid_1's l2: 0.426258
[600]	training's l2: 0.37936	valid_1's l2: 0.426082
[650]	training's l2: 0.379026	valid_1's l2: 0.425971
[700]	training's l2: 0.378716	valid_1's l2: 0.425833
[750]	training's l2: 0.37843	valid_1's l2: 0.425698
[800]	training's l2: 0.378147	valid_1's l2: 0.42564
[850]	training's l2: 0.377888	valid_1's l2: 0.425587
[900]	training's l2: 0.377621	valid_1's l2

In [57]:
print("Making submission...")
y_test = np.array(test_pred).transpose()
df_preds = pd.DataFrame(
    y_test, index=df_train.index,
    columns=pd.date_range("2017-08-16", periods=16)
).stack().to_frame("unit_sales")
df_preds.index.set_names(["store_nbr", "item_nbr", "date"], inplace=True)

submission = df_test[["id"]].join(df_preds, how="left").fillna(0)
submission["unit_sales"] = np.clip(np.expm1(submission["unit_sales"]), 0, 1000)
submission.to_csv('modified_weight_promo.csv', float_format='%.4f', index=None)

Making submission...
